**Implementación de una Máquina de Estados Finito (FSM) Interactiva** 

Este programa implementa un reconocedor de entradas de un usuario basado en una FSM predefinido. Requiere que el diseñador especifique la *tabla de transición* del FA. La entrada se reconoce mediante un ASR y la salida se genera mediante la reutilización de un TTS.

Primero, necesitamos instalar algunos paquetes para ASR, TTS y manejo de audio:

In [1]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install speechrecognition
!pip install PyAudio
!pip -q install pydub
!pip install gtts
!pip install pygobject
!pip install spacy
!python -m spacy download es_core_news_sm

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.6).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 20 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (128 kB/s)
Selecting p

Luego importamos algunas bibliotecas que necesitaremos:

In [2]:
import speech_recognition as sr 
from pydub import AudioSegment
from IPython.display import Javascript, Audio
from google.colab import output
from base64 import b64decode
from io import BytesIO
import subprocess
import re
from gtts import gTTS 
import time
import es_core_news_sm

Luego, re-utilizamos algunos métodos definidos previamente para ASR y TTS:

In [3]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def Grabar(sec=5):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.webm','wb') as f:
    f.write(b)
  command = ['ffmpeg', '-i', 'audio.webm', '-f', 'segment', '-segment_time', '100', 'out%02d.wav']
  subprocess.run(command,stdout=subprocess.PIPE,stdin=subprocess.PIPE)
  return('out00.wav')

def ASR():
    texto_reconocido=""
    print("Comenzar..")
    WAV = Grabar()
    fuente = sr.AudioFile(WAV)
    r = sr.Recognizer()
    with fuente as source:
         audio = r.record(source)                                 
    try:
         texto_reconocido = r.recognize_google(audio, language='es-es')
    except sr.UnknownValueError:
       print("No entendi el audio")
    return(texto_reconocido)

def TTS(texto):
  sound_file = 'aux.wav'
  tts = gTTS(texto, lang="es")
  tts.save(sound_file) 
  display(Audio(sound_file,autoplay=True)) 

Además, re-utilizamos nuestra función para lematizar una oración:

In [4]:
def Lematizar(oracion):
   doc = nlp(oracion)
   lemas = [token.lemma_ for token in doc]
   return(" ".join(lemas))  

Ahora, definimos el método **LeerRespuesta(pregunta, patron)**, que lee una respuesta hablada del usuario a partir de una **pregunta** dada por el FA. En el caso de que la respuesta "calce" con algunos de los símbolos permitidos en **patron** (i.e., símbolos de *Sigma*), se retorna el calce:

In [5]:
def LeerRespuesta(pregunta,patron):
        # Sintetizar voz de la pregunta
        TTS(pregunta)
        time.sleep(4)
        # Reconocer respuesta hablada
        texto = ASR()
        # Probar que la frase hablada contenga alguno de los símbolos del patron
        calce = re.search(patron,texto)
        if calce != None:
           return(calce.group(0))
        else:
           return("nada")

Luego, definimos el reconocedor basado en el FA con la función **FA(q0,F,Sigma,Preguntas,TablaTrans)**. Esta recibe el estado inicial (**Q0**) y final (**F**), el vocabulario **Sigma**, la lista de **Preguntas** para cada estado, y la tabla de transición **TablaTrans** que el FA debe recorrer. La función retorna *True* si la entrada completa se reconoce, o *False*, en caso contrario.

Recuerde que el lenguaje a reconocer es el dado por el siguiente autómata de control de flujo:

<img src="https://drive.google.com/uc?id=1C-0X7inkaOesCbNqZFbYb50qY3ZNF38k" width="500" > 

El algoritmo que implementa un FA, dada una tabla de transición, es el siguiente:

* *q = estado inicial*
* *F = conjunto de estados finales (podría haber más de uno)*
* *Repita mientras q no está en F  o no sea un estado de error:*
   * *Simbolo = Entrada leida del usuario*
   * *(Actualizar el estado)*
   * *q = TablaTransicion[q][Simbolo]*
   * *(Validar que el simbolo exista para el estado actual)*   
* *Fin repetición*
* *Si q está en F Entonces retorne TRUE (reconoce)*
* *Sino retorne FALSE (no reconoce)*


In [6]:
def FA(Q0,F,Sigma,Preguntas,TablaTrans):
    q=Q0    # Estado inicial
    patron = "|".join(Sigma)
    # Repita mientras no sea un estado final o de error
    while ( not(q in F)   and  (q != _ERROR)):
        # En el caso de que desee leer la respuesta solamente desde teclado,
        # reemplace la línea de abajo por: Sym = input(Preguntas[q])
        Simbolo  = LeerRespuesta(Preguntas[q],patron)
        try:
           TablaTrans[q][Simbolo]
        except KeyError:
           q = _ERROR
           break
        # Asigne el siguiente estado de la tabla o bien ERROR si no existe
        q = (_ERROR if (not(Simbolo in Sigma)) else TablaTrans[q][Simbolo])
    return(q in F)

Necesitamos, una función que inicialice y defina los estados del FA, por lo que definimos la función **InicializarFA(nQ,Sigma)**, que toma el número de estados **nQ** de la máquina y el vocabulario **Sigma**, e inicializa los estados de la tabla de transición, según el grafo de interacción especificado en clases. La función retorna la tabla de transición llena (por defecto, los estados inválidos se llenan con un valor de **ERROR**):

In [7]:
def InicializarFA(nQ,Sigma):
   tt = {}
   # Crear tabla de transición vacia (con valor de error)
   for numQ in range (nQ):
      tt[numQ] = {}
      for Simb in Sigma:
         tt[numQ][Simb] = _ERROR
   # LLenar transiciones no vacias
   tt[0]['internet']        = 1
   tt[0]['teléfono']        = 2
   tt[1]['tarjeta']         = 1
   tt[1]['mensual']         = 3
   tt[2]['casa']            = 3
   tt[2]['departamento']    = 1
   return(tt)

Ahora definimos la función **EspecificarPreguntas()**, que simplemente especifica las preguntas que el FA realizará al usuario en cada estado, retornando la lista de preguntas posibles (existe una pregunta por cada estado, exceptuando el final): 

In [8]:
def EspecificarPreguntas():
   Preguntas = ["Qué servicio requiere? ",
                "¿Qué tipo de cliente? ",
                "¿En qué residencia? "
            ]
   return(Preguntas)

Ahora, invocamos el programa principal:

In [ ]:
# Cargamos modelo de lenguaje en Español
nlp       = es_core_news_sm.load()
# Estado de error
_ERROR=(-1)
# Numero de estados   
nQ=4    
# Vocabulario (alfabeto) de la máquina:
Sigma = {'internet','teléfono','tarjeta','mensual','casa','departamento'}
q0=0    # Estado inicial
F={3}   # Estados finales
TablaTransicion = InicializarFA(nQ,Sigma)
TTS("Bienvenido a MasterPlop Comunicaciones..")
time.sleep(5)
Preguntas  = EspecificarPreguntas()
status = FA(q0,F,Sigma,Preguntas,TablaTransicion)
if (status):
    TTS("Transacción aceptada!")
else:
    TTS("Error, no se entendió la consulta!")

Comenzar..


<IPython.core.display.Javascript object>